In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline  # NOTE: imblearn pipeline used because of SMOTE

In [2]:
# Load the data
df = pd.read_csv('C:\\Users\\91964\\Desktop\\csp\\customer_support_tickets.csv')

In [3]:
# Separate features and target
X = df.drop('Customer Satisfaction Rating', axis=1)
y = df['Customer Satisfaction Rating']

In [4]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
# Define preprocessing for numeric and categorical columns
numeric_features = ['Customer Age']
numeric_transformer = StandardScaler()

categorical_features = ['Customer Gender', 'Product Purchased', 'Ticket Type',
                        'Ticket Status', 'Ticket Priority', 'Ticket Channel',
                        'Viewed Ticket', 'Ticket Solved']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [6]:
# SMOTE for class imbalance
smote = SMOTE(random_state=42)

In [7]:
# Pipeline with RandomForestClassifier (basic model)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', smote),
    ('classifier', RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        class_weight='balanced'
    ))
])

In [8]:
# Fit model
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Customer Age']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Customer Gender',
                                                   'Product Purchased',
                                                   'Ticket Type',
                                                   'Ticket Status',
                                                   'Ticket Priority',
                                                   'Ticket Channel',
                                                   'Viewed Ticket',
                                                   'Ticket Solved'])])),
                ('smote', SMOTE(random_state=42)),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=15,
                                        min_samples_leaf=2, min_samples_split=5,
                                        n_estimators=200, random_state=42))])

In [9]:
# Predict
y_pred = model.predict(X_test)

In [10]:
# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.73

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1140
           1       0.17      0.16      0.17       111
           2       0.12      0.15      0.14       110
           3       0.22      0.22      0.22       116
           4       0.18      0.18      0.18       108
           5       0.26      0.23      0.24       109

    accuracy                           0.73      1694
   macro avg       0.33      0.32      0.32      1694
weighted avg       0.74      0.73      0.73      1694



In [11]:
# Feature names after encoding
feature_names = numeric_features + list(
    model.named_steps['preprocessor']
         .named_transformers_['cat']
         .get_feature_names_out(categorical_features)
)

In [12]:
# Feature importances
importances = model.named_steps['classifier'].feature_importances_
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importances = feature_importances.sort_values('importance', ascending=False)

print("\nTop 10 Important Features:")
print(feature_importances.head(10))


Top 10 Important Features:
                 feature  importance
63       Ticket Solved_0    0.125078
64       Ticket Solved_1    0.118104
52       Ticket Status_2    0.108287
0           Customer Age    0.058695
51       Ticket Status_1    0.043246
62       Viewed Ticket_1    0.037543
61       Viewed Ticket_0    0.033906
50       Ticket Status_0    0.030596
58  Ticket Channel_Email    0.021249
54     Ticket Priority_1    0.019969


In [13]:
# ---------- Optimized model with XGBoost ----------

# XGBoost pipeline
optimized_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', smote),
    ('classifier', XGBClassifier(
        objective='multi:softmax',
        num_class=6,
        random_state=42,
        eval_metric='mlogloss',
        use_label_encoder=False
    ))
])

In [14]:
# Hyperparameter tuning
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.01, 0.1, 0.2],
    'classifier__subsample': [0.8, 1.0],
    'classifier__colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(optimized_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

C:\Users\91964\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
4 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\91964\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\91964\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\91964\AppData\Roaming\Python\Python312\site-pa

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Customer '
                                                                          'Age']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Customer '
                                                                          'Gender',
                                                                          'Product '
                                                                          'Purchased',
                                                                          'Ticket '
                                                                          'Type',
                                                                          'Ticket '
                                                                          'Status',
                                                                          'Ticket '
                                                                          'Priority',
                                                                          'Ticket '
                                                                          'Channel',
                                                                          'Viewed '
                                                                          'Ticket',
                                                                          'Ticket '
                                                                          'Solved'])])),
                                       ('smote', SMOTE(rando...
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None, num_class=6, ...))]),
             n_jobs=-1,
             param_grid={'classifier__colsample_bytree': [0.8, 1.0],
                         'classifier__learning_rate': [0.01, 0.1, 0.2],
                         'classifier__max_depth': [3, 5, 7],
                         'classifier__n_estimators': [100, 200, 300],
                         'classifier__subsample': [0.8, 1.0]},
             scoring='accuracy')

In [15]:
# Best model
best_model = grid_search.best_estimator_

In [16]:
# Predict and evaluate
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized Model Accuracy: {accuracy:.2f}")

Optimized Model Accuracy: 0.74


In [17]:
# Predict on new data
new_ticket = pd.DataFrame({
    'Customer Age': [35],
    'Customer Gender': [1],
    'Product Purchased': ['Microsoft Office'],
    'Ticket Type': ['Billing inquiry'],
    'Ticket Status': [2],
    'Ticket Priority': [0],
    'Ticket Channel': ['Email'],
    'Viewed Ticket': [1],
    'Ticket Solved': [1]
})

prediction = best_model.predict(new_ticket)
print(f"Predicted Satisfaction: {prediction[0]}")

Predicted Satisfaction: 4
